In [ ]:
import psycopg2
import pandas as pd
from rapidfuzz import process, fuzz

# Koneksi ke database PostgreSQL Anda
conn = psycopg2.connect(
    dbname="faztream_db",
    user="postgres",      # ganti sesuai user Anda
    password="admin123",  # ganti sesuai password Anda
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# List semua tabel genre
# genre_tables = [
#     "action", "adventure", "animation", "anime", "biography", "comedy", "crime", "drama",
#     "family", "fantasy", "horror", "mystery", "romance", "science_fiction", "sport", "talk", "thriller"
# ]

# khusus genre myster
genre_tables = ["mystery"]

# Ambil semua title dari movies
cur.execute("SELECT id, title FROM movies")
movies = cur.fetchall()
movie_titles = [row[1] for row in movies]

for genre in genre_tables:
    print(f"Processing {genre}...")
    cur.execute(f"SELECT title FROM {genre}")
    genre_titles = [row[0] for row in cur.fetchall()]

    for g_title in genre_titles:
        match, score, idx = process.extractOne(g_title, movie_titles, scorer=fuzz.token_sort_ratio)
        # Jika skor cukup tinggi (misal >= 85), lakukan update
        if score >= 65 and match != g_title:
            cur.execute(
                f"UPDATE {genre} SET title = %s WHERE title = %s",
                (match, g_title)
            )
            print(f"Updated {genre}: '{g_title}' -> '{match}' (score: {score})")

conn.commit()
cur.close()
conn.close()
print("Selesai update semua genre!")

Processing mystery...
Selesai update semua genre!
